In [2]:
import pandas as pd
import csv
from demyst.analytics import Analytics


analytics = Analytics(username="apac.candidate.bruceli@demystdata.com", password="1qaz@WSX3edc")
#read file
inputs = pd.read_csv('input_file.csv', encoding="ISO-8859-1")


# 1. clean and enrich data

## a. Clean data - 1st Time
    Initially, clean the data according to analytics.validate(inputs)

In [3]:
inputs.dropna(inplace=True)
# change postcode to the suggested name of analytics.validate
inputs.rename(columns={"postcode":"post_code"}, inplace=True)
# clean white spaces
inputs['state'] = inputs['state'].str.strip()
# attom_expanded_profile_report payload need country to get response
inputs['country'] = 'US'

In [4]:
#validate input data
analytics.validate(inputs)

## b. Enrich Data - 1st Time

In [5]:
# search related data provider 
analytics.search(inputs)


,state,street,country,city,post_code
Option 1,☒,☒,☒,☒,☒


In [6]:
# get small sample inputs data 
sample_inputs = inputs.sample(n=1000, random_state=0)
sample_inputs.head()

,street,state,city,post_code,safety_flag,country
6197,2134 Lincoln Park Drive,MI,Muskegon,49441,False,US
7974,222 Robert Lane,TX,El Campo,77437,False,US
2153,300 Affeld Lane,TX,Smithville,78957,False,US
3284,115 Private Road 2131,TX,Iredell,76649-4602,False,US
6511,6570 Lexi Ln Apt 102,NC,Fayetteville,28314-0443,False,US


In [56]:
# send sample data back to server and get enriched
enrich_job_id = analytics.enrich(['attom_expanded_profile_report'], sample_inputs, validate=False)


Verifying providers...
Starting enrichment...
Uploading data...


This enrichment will use 2700 credits of the 17161 credits your organization currently has.


Enrich Job ID: 6137


In [8]:
# check the availability of the enriched data
# delete this row if you want try get new data for enrichment
enrich_job_id = 6137

analytics.enrich_status(enrich_job_id)

attom_expanded_profile_report (done)


True

In [9]:
# download enriched data
results = analytics.enrich_download(enrich_job_id, block_until_complete=False)

results.head()

,inputs.city,inputs.country,inputs.post_code,inputs.safety_flag,inputs.state,inputs.street,attom_expanded_profile_report.row_id,attom_expanded_profile_report.client_id,attom_expanded_profile_report.attom_id,attom_expanded_profile_report.error
0,Muskegon,US,49441,False,MI,2134 Lincoln Park Drive,0,,172292268,
1,El Campo,US,77437,False,TX,222 Robert Lane,1,,,
2,Smithville,US,78957,False,TX,300 Affeld Lane,2,,8328494,
3,Iredell,US,76649-4602,False,TX,115 Private Road 2131,3,,216350563,
4,Fayetteville,US,28314-0443,False,NC,6570 Lexi Ln Apt 102,4,,,


## 3. Clean Data- 2nd Time
    clean the enriched data, and then clean data again preparing for enrich more features  

In [10]:
# reanme the column of the results data prepare for analytics.search
results.columns = results.columns.map(lambda x : x.split('.')[1])
results.head()

,city,country,post_code,safety_flag,state,street,row_id,client_id,attom_id,error
0,Muskegon,US,49441,False,MI,2134 Lincoln Park Drive,0,,172292268,
1,El Campo,US,77437,False,TX,222 Robert Lane,1,,,
2,Smithville,US,78957,False,TX,300 Affeld Lane,2,,8328494,
3,Iredell,US,76649-4602,False,TX,115 Private Road 2131,3,,216350563,
4,Fayetteville,US,28314-0443,False,NC,6570 Lexi Ln Apt 102,4,,,


In [11]:
# drop unnecessary columns
results.drop(columns=['row_id', 'client_id', 'error'], inplace=True)

In [12]:
sum(results.duplicated())

0

In [13]:
#check empty cell
sum(results['attom_id']=='')

204

In [14]:
sum(results['attom_id'].isna())

0

In [15]:
# check null value
sum(results['attom_id'].isnull())

0

In [16]:
outputs = results[results['attom_id']!='']

In [17]:
# validate data preparing for searching more related data
analytics.validate(outputs)

In [103]:
outputs.to_dense().to_csv('outputs.csv', index = False, sep=',', encoding='utf-8')


## 4. Enrich data - 2nd Time

In [104]:
analytics.search(outputs)


In [105]:
# by testing with small data by analytics.validate function, 
#'attom_commercial_avm' is for commercial and is less useful than attom_avm
new_enrich_job_id = analytics.enrich(['attom_avm'], outputs, validate=False)


Verifying providers...
Starting enrichment...
Uploading data...


This enrichment will use 2149.2 credits of the 14461 credits your organization currently has.


Enrich Job ID: 6138


In [19]:
#Enrich Job ID: 6138
# delete this row if you want try get new data for enrichment
new_enrich_job_id = 6138

analytics.enrich_status(new_enrich_job_id)

attom_avm (done)


True

In [20]:
results2 = analytics.enrich_download(new_enrich_job_id, block_until_complete=False)


In [21]:
results2.head()

,inputs.attom_id,inputs.city,inputs.country,inputs.post_code,inputs.safety_flag,inputs.state,inputs.street,attom_avm.row_id,attom_avm.client_id,attom_avm.attom_id,attom_avm.confidencescore,attom_avm.createdate,attom_avm.estimatedmaxvalue,attom_avm.estimatedminvalue,attom_avm.estimatedvalue,attom_avm.fsd,attom_avm.lastupdatedate,attom_avm.publicationdate,attom_avm.valuationdate,attom_avm.error
0,172292268,Muskegon,US,49441,False,MI,2134 Lincoln Park Drive,0,,172292268,70,2019-05-08,185000.0,126000.0,155000.0,19,2019-05-08,2019-05-23,2019-04-29,
1,8328494,Smithville,US,78957,False,TX,300 Affeld Lane,1,,,,,,,,,,,,
2,216350563,Iredell,US,76649-4602,False,TX,115 Private Road 2131,2,,,,,,,,,,,,
3,36125343,Charleston,US,29414,False,SC,7106 Windmill Creek Road,3,,36125343,91,2019-05-08,310000.0,251000.0,281000.0,10.0,2019-05-08,2019-05-23,2019-04-29,
4,4070570,Little Elm,US,75068-6309,False,TX,1937 Caddo St,4,,4070570,93,2019-05-08,274000.0,227000.0,251000.0,9,2019-05-08,2019-05-23,2019-04-29,


## 4 Clean data - Last step
    Clean data for output and analyzing purpose

In [22]:
#get columns
results2.columns

Index(['inputs.attom_id', 'inputs.city', 'inputs.country', 'inputs.post_code',
       'inputs.safety_flag', 'inputs.state', 'inputs.street',
       'attom_avm.row_id', 'attom_avm.client_id', 'attom_avm.attom_id',
       'attom_avm.confidencescore', 'attom_avm.createdate',
       'attom_avm.estimatedmaxvalue', 'attom_avm.estimatedminvalue',
       'attom_avm.estimatedvalue', 'attom_avm.fsd', 'attom_avm.lastupdatedate',
       'attom_avm.publicationdate', 'attom_avm.valuationdate',
       'attom_avm.error'],
      dtype='object')

In [23]:
results2 = results2[['inputs.attom_id','inputs.city', 
             'inputs.country', 'inputs.state',
             'inputs.post_code',
              'attom_avm.confidencescore', 
             'attom_avm.estimatedvalue', 'attom_avm.fsd',
             'inputs.safety_flag']]


In [24]:
# reanme the column of the results data prepare for analytics.search
results2.columns = results2.columns.map(lambda x : x.split('.')[1])
results2.head()

,attom_id,city,country,state,post_code,confidencescore,estimatedvalue,fsd,safety_flag
0,172292268,Muskegon,US,MI,49441,70,155000.0,19,False
1,8328494,Smithville,US,TX,78957,,,,False
2,216350563,Iredell,US,TX,76649-4602,,,,False
3,36125343,Charleston,US,SC,29414,91,281000.0,10.0,False
4,4070570,Little Elm,US,TX,75068-6309,93,251000.0,9,False


In [30]:
outputs2 = results2[(results2['confidencescore']!='') &
                    (results2['estimatedvalue']!='') &
                    (results2['fsd']!='')
                   ]

In [32]:

outputs2.head()

,attom_id,city,country,state,post_code,confidencescore,estimatedvalue,fsd,safety_flag
0,172292268,Muskegon,US,MI,49441,70,155000.0,19,False
3,36125343,Charleston,US,SC,29414,91,281000.0,10.0,False
4,4070570,Little Elm,US,TX,75068-6309,93,251000.0,9,False
5,18937891,Winthrop,US,MA,2152,68,508000.0,20.0,False
6,2316243,Milwaukee,US,WI,53217-4732,86,405000.0,12,False


In [148]:
outputs2.to_dense().to_csv('outputs2.csv', index = False, sep=',', encoding='utf-8')
